# Simulation for non-contextual settings
We adjusted the simulation file provided by [Hadad, Vitor, et al. (2021)](https://arxiv.org/abs/1911.02768) to only contains the methods of interest in our banditsCI package: uniform and two-point.

The original simulation code provided by [Hadad, Vitor, et al. (2021)](https://arxiv.org/abs/1911.02768) can be accessed [here](https://github.com/gsbDBI/adaptive-confidence-intervals/blob/master/experiments/main/simulations.ipynb).

In [1]:

import sys
import subprocess
import pickle
import os
import numpy as np
import pandas as pd

from time import time
from sys import argv
from random import choice
from time import time
from os.path import dirname, realpath, join, exists
from os import makedirs, chmod
from getpass import getuser

from adaptive_CI.experiments import run_mab_experiment
from adaptive_CI.compute import stick_breaking
from adaptive_CI.inference import *
from adaptive_CI.weights import *

%reload_ext autoreload
%autoreload 2

In [2]:
start_time = time()

In [3]:
def on_sherlock():
    """ 
    Note: This can be ignored by non-Stanford members.

    Checks if running on Stanford's Sherlock cluster
    """
    return 'GROUP_SCRATCH' in os.environ


def get_sherlock_dir(project, *tail, create=True):
    """
    Note: This can be ignored by non-Stanford members.
    
    Output consistent folder name in Sherlock.
    If create=True and on Sherlock, also makes folder with group permissions.
    If create=True and not on Sherlock, does not create anything.

    '/scratch/groups/athey/username/project/tail1/tail2/.../tailn'.

    >>> get_sherlock_dir('adaptive-inference')
    '/scratch/groups/athey/adaptive-inference/vitorh'

    >>> get_sherlock_dir('toronto')
    '/scratch/groups/athey/toronto/vitorh/'

    >>> get_sherlock_dir('adaptive-inference', 'experiments', 'exp_out')
    '/scratch/groups/athey/adaptive-inference/vitorh/experiments/exp_out'
    """
    base = join("/", "scratch", "groups", "athey", project, getuser())
    path = join(base, *tail)
    if not exists(path) and create and on_sherlock():
        makedirs(path, exist_ok=True)
        # Correct permissions for the whole directory branch
        chmod_path = base
        chmod(base, 0o775)
        for child in tail:
            chmod_path = join(chmod_path, child)
            chmod(chmod_path, 0o775)
    return path


def compose_filename(prefix, extension):
    """
    Creates a unique filename based on Github commit id and time.
    Useful when running in parallel on server.

    INPUT:
        - prefix: file name prefix
        - extension: file extension

    OUTPUT:
        - fname: unique filename
    """
    # Tries to find a commit hash
    try:
        commit = subprocess\
            .check_output(['git', 'rev-parse', '--short', 'HEAD'],
                          stderr=subprocess.DEVNULL)\
            .strip()\
            .decode('ascii')
    except subprocess.CalledProcessError:
        commit = ''

    # Other unique identifiers
    rnd = str(int(time() * 1e8 % 1e8))
    sid = tid = jid = ''
    ident = filter(None, [prefix, commit, jid, sid, tid, rnd])
    basename = "_".join(ident)
    fname = f"{basename}.{extension}"
    return fname

In [4]:
num_sims = 200 if on_sherlock() else 1

# DGP specification
# ----------------------------------------------------
noise_func = 'uniform'
truths = {
    'nosignal': np.array([1., 1., 1.]),
    'lowSNR': np.array([.9, 1., 1.1]),
    'highSNR': np.array([.5, 1., 1.5])
}

Ts = [5_000]
floor_decays = [.7]
initial = 5  # initial number of samples of each arm to do pure exploration
exploration = 'TS'
noise_scale = 1.

In [5]:
df_stats = []
df_lambdas = []

## Simulations

In [6]:
# Run simulations
for s in range(num_sims):
    if (s+1) % 10 == 0:
        print(f'Running simulation {s+1}/{num_sims}')

    """ Experiment configuration """
    T = choice(Ts)  # number of samples
    experiment = choice(list(truths.keys()))
    truth = truths[experiment]
    K = len(truth)  # number of arms
    floor_start = 1/K
    floor_decay = choice(floor_decays)

    """ Generate data """
    noise = np.random.uniform(-noise_scale, noise_scale, size=(T, K))
    ys = truth + noise

    """ Run experiment """
    data = run_mab_experiment(
        ys,
        initial=initial,
        floor_start=floor_start,
        floor_decay=floor_decay,
        exploration=exploration)

    probs = data['probs']
    rewards = data['rewards']
    arms = data['arms']

    """ Compute AIPW scores """
    muhat = np.row_stack([np.zeros(K), sample_mean(rewards, arms, K)[:-1]])
    scores = aw_scores(rewards, arms, probs, muhat)

    """ Compute weights """
    # Two-point allocation rate
    twopoint_ratio = twopoint_stable_var_ratio(e=probs, alpha=floor_decay)
    twopoint_ratio_old = twopoint_stable_var_ratio_old(probs, floor_start, floor_decay)
    twopoint_h2es = stick_breaking(twopoint_ratio)
    twopoint_h2es_old = stick_breaking(twopoint_ratio_old)
    wts_twopoint = np.sqrt(np.maximum(0., twopoint_h2es * probs))

    # Other weights: lvdl(constant allocation rate), propscore and uniform
    wts_lvdl = np.sqrt(probs)
    wts_propscore = probs
    wts_uniform = np.ones_like(probs)

    """ Estimate arm values """
    # for each weighting scheme, return [estimate, S.E, bias, 90%-coverage, t-stat, mse, truth]
    stats = dict(
        uniform=evaluate_aipw_stats(scores, wts_uniform, truth),
        propscore=evaluate_aipw_stats(scores, wts_propscore, truth),
        lvdl=evaluate_aipw_stats(scores, wts_lvdl, truth),
        two_point=evaluate_aipw_stats(scores, wts_twopoint, truth),
        beta_bernoulli=evaluate_beta_bernoulli_stats(rewards, arms, truth, K, floor_decay, alpha=.1),
        gamma_exponential=evaluate_gamma_exponential_stats(rewards, arms, truth, K, floor_decay, c=2, expected_noise_variance=1/3, alpha=.1),
        sample_mean_naive=evaluate_sample_mean_naive_stats(rewards, arms, truth, K, alpha=.1)
    )

    
    # # add estimates of W_decorrelation
    W_name = f'wdecorr_results/W_lambdas_{experiment}-{noise_func}-{T}-{floor_decay}.npz'
    try:
        W_save = np.load(W_name)  # load presaved W-lambdas
        for percentile, W_lambda in zip(W_save['percentiles'], W_save['W_lambdas']):
            stats[f'W-decorrelation_{percentile}'] = wdecorr_stats(arms, rewards, K, W_lambda, truth)
    except FileNotFoundError:
        print(f'Could not find relevant w-decorrelation file {W_name}. Ignoring.')
        
    
    """ Estimate contrasts """
    contrasts = dict(
        uniform=evaluate_aipw_contrasts(scores, wts_uniform, truth),
        propscore=evaluate_aipw_contrasts(scores, wts_propscore, truth),
        lvdl=evaluate_aipw_contrasts(scores, wts_lvdl, truth),
        two_point=evaluate_aipw_contrasts(scores, wts_twopoint, truth),
        beta_bernoulli=evaluate_beta_bernoulli_contrasts(rewards, arms, truth, K, floor_decay, alpha=.1),
        gamma_exponential=evaluate_gamma_exponential_contrasts(rewards, arms, truth, K, floor_decay, c=2, expected_noise_variance=1/3, alpha=.1),
        sample_mean_naive=evaluate_sample_mean_naive_contrasts(rewards, arms, truth, K, alpha=.1)
    )
    # Print the two_point value
    
    """ Save results """
    config = dict(
        T=T,
        K=K,
        noise_func=noise_func,
        noise_scale=noise_scale,
        floor_start=floor_start,
        floor_decay=floor_decay,
        initial=initial,
        dgp=experiment,
    )

    ratios = dict(
        lvdl=np.ones((T, K)) / np.arange(T, 0, -1)[:, np.newaxis],
        two_point=twopoint_ratio,
    )
    
    # save lambda values at selected timepoints
    saved_timepoints = list(range(0, T, T // 250)) + [T-1]
    for ratio in ratios:
        ratios[ratio] = ratios[ratio][saved_timepoints, :]
    
    # tabulate arm values
    tabs_stats = []
    for method, stat in stats.items():
        tab_stats = pd.DataFrame({"statistic": ["estimate", "stderr", "bias", "90% coverage of t-stat", "t-stat", "mse", "CI_width", "truth"] * stat.shape[1],
                                  "policy": np.repeat(np.arange(K), stat.shape[0]),
                                  "value":  stat.flatten(order='F'),
                                  "method": method,
                                 **config})
        tabs_stats.append(tab_stats)


    # tabulate arm contrasts
    tabs_contrasts = []
    for method, contrast in contrasts.items():
        tabs_contrast = pd.DataFrame({"statistic": ["estimate", "stderr", "bias", "90% coverage of t-stat", "t-stat", "mse", "CI_width", "truth"] * contrast.shape[1],
                                      "policy": np.repeat([f"(0,{k})" for k in np.arange(1, K)], contrast.shape[0]),
                                      "value": contrast.flatten(order='F'),
                                      "method": method,
                                     **config})
        tabs_contrasts.append(tabs_contrast)

    
    df_stats.extend(tabs_stats)
    df_stats.extend(tabs_contrasts)
    
    
    """ Save relevant lambda weights, if applicable """
    if T == max(Ts):
        saved_timepoints = list(range(0, T, T // 500))
        lambdas = twopoint_ratio[saved_timepoints] * (T - np.array(saved_timepoints)[:,np.newaxis])
        lambdas = {key: value for key, value in enumerate(lambdas.T)}
        dfl = pd.DataFrame({**lambdas, **config, 'time': saved_timepoints})
        dfl = pd.melt(dfl, id_vars=list(config.keys()) + ['time'], var_name='policy', value_vars=list(range(K)))
        df_lambdas.append(dfl)
        
    print(f"Time passed {time()-start_time}s")

Time passed 0.3952500820159912s


/var/folders/vw/_ffz0lz95v93q6_6s4x_60840000gp/T/ipykernel_72617/4042344571.py:31: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  muhat = np.row_stack([np.zeros(K), sample_mean(rewards, arms, K)[:-1]])


----

### Estimates of mean response under each of the treatment arms

In [7]:
# Concatenate all DataFrames in the `df_stats` list into one DataFrame
df_stats = pd.concat(df_stats, ignore_index=True)

# for uniform method
df_stats.loc[(df_stats['method'] == "uniform") & (df_stats['statistic'].isin(['estimate','stderr']))]

,statistic,policy,value,method,T,K,noise_func,noise_scale,floor_start,floor_decay,initial,dgp
0,estimate,0,0.965982,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
1,stderr,0,0.025985,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
8,estimate,1,1.009563,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
9,stderr,1,0.012324,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
16,estimate,2,1.019343,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
17,stderr,2,0.017162,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
264,estimate,"(0,1)",0.053361,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
265,stderr,"(0,1)",0.031140,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
272,estimate,"(0,2)",0.009781,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
273,stderr,"(0,2)",0.021129,uniform,5000,3,uniform,1.0,0.333333,0.7,5,nosignal


In [8]:
# for two-point method
df_stats.loc[(df_stats['method'] == "two_point") & (df_stats ['statistic'].isin(['estimate','stderr']))]

,statistic,policy,value,method,T,K,noise_func,noise_scale,floor_start,floor_decay,initial,dgp
72,estimate,0,0.973298,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
73,stderr,0,0.025012,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
80,estimate,1,1.006657,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
81,stderr,1,0.012950,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
88,estimate,2,1.014480,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
89,stderr,2,0.017365,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
312,estimate,"(0,1)",0.041182,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
313,stderr,"(0,1)",0.030516,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
320,estimate,"(0,2)",0.007823,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal
321,stderr,"(0,2)",0.021644,two_point,5000,3,uniform,1.0,0.333333,0.7,5,nosignal


### Saving

Save the `scores` and `probs` generated from the experiment. We will use them with banditsCI R package to compare the estimates and standard errors

In [9]:
np.savetxt("./results/scores.csv", scores, delimiter=",")
np.savetxt("./results/probs.csv", probs, delimiter=",")

In [10]:
write_dir = join(os.getcwd(), 'results')

Saving information about contrasts.

In [11]:
filename_contrast = compose_filename(f'contrast', 'pkl')
write_path_contrast = os.path.join(write_dir, filename_contrast)

df_contrast = df_stats.query(
    '(policy == "(0,1)" or policy == "(0,2)") and '
    'statistic in ["estimate", "stderr", "mse", "bias", "CI_width"] and '
    "method in ['uniform', 'two_point', 'sample_mean_naive']")
df_contrast.to_pickle(write_path_contrast)